In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del_400_3_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1050025

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

7797215

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700308941
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 52501171


b'chr22\t396\tT\tW\t+\nchr22\t2304\tG\t-\t-\nchr22\t3145\tC\tM\t+\nchr22\t3231\tC\tA\t-\nchr22\t3257\t-\tC\t+\nchr22\t4380\tG\tA\t-\nchr22\t5341\tT\tG\t-\nchr22\t5707\tA\tW\t+\nchr22\t6150\tA\tM\t+\nchr22\t6764\tG\tS\t+\nchr22\t7437\tA\tR\t+\nchr22\t8006\t-\tT\t+\nchr22\t8034\t-\tA\t+\nchr22\t8825\tT\tG\t-\nchr22\t9712\tA\tM\t+\nchr22\t11468\tT\tK\t+\nchr22\t11920\tT\tG\t-\nchr22\t13320\tA\tM\t+\nchr22\t14930\tG\tS\t+\nchr22\t15032\tA\tM\t+\nchr22\t16249\tG\tR\t+\nchr22\t16814\tA\tT\t-\nchr22\t19836\tC\tT\t-\nchr22\t20189\tG\tC\t-\nchr22\t20524\tG\tT\t-\nchr22\t21757\tC\tY\t+\nchr22\t24466\tA\tG\t-\nchr22\t25712\tC\tT\t-\nchr22\t26838\tG\tS\t+\nchr22\t28661\tT\tY\t+\nchr22\t29344\tA\tW\t+\nchr22\t29705\tT\tC\t-\nchr22\t29893\tG\tS\t+\nchr22\t30003\tC\tT\t-\nchr22\t30989\tA\t-\t-\nchr22\t34252\t-\tGT\t+\nchr22\t34834\tT\tK\t+\nchr22\t34942\tA\tW\t+\nchr22\t35003\tA\tR\t+\nchr22\t35157\t-\tC\t+\nchr22\t35929\tC\tY\t+\nchr22\t38019\tC\tA\t-\nchr22\t38209\tA\tT\t-\nchr22\t39756\tG\tR\t+\nc

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 9 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.7185	1.186329e-06	2.284863e-87	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.5587	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:2926301-2963000	36700	1.8470	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655101-6661800	6700	1.5677	0.000000e+00	2.454044e-181	0.000000e+00	5.251180e-226	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	2.4458	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13480601-13513300	32700	6.0506	0.000000e+00	6.984599e-19	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:15242601-15246200	3600	1.8349	0.000000e+00	2.782450e-239	2.051921e-08	1.116883e-116	0.0000	1.0000	0
duplication	chr22:17447001-17639900	192900	1.2797	0.000000e+00	2.205830e-195	0.000000e+00	2.882998e-193	0.0001	0.0000	599300
duplication	chr22:17641801-17810700	168900	1.2764	0.000000e+00	4.307

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.718514374302551,0.0
duplication,chr22,1306101,1325100,19000,1.558743594894439,0.0
duplication,chr22,2926301,2963000,36700,1.8470098672244164,0.0
duplication,chr22,6655101,6661800,6700,1.567712576785245,0.0
duplication,chr22,8039401,8047900,8500,2.445822970635835,0.0
duplication,chr22,13480601,13513300,32700,6.0506295711806475,0.0
duplication,chr22,15242601,15246200,3600,1.834909360819113,0.0
duplication,chr22,17447001,17639900,192900,1.2796932767012235,8.353695466031786e-05
duplication,chr22,17641801,17810700,168900,1.2763572110123276,8.255844268706006e-05
duplication,chr22,18662501,18665900,3400,1.851143694198247,0.0
duplication,chr22,18680401,18682700,2300,1.7209608810822625,0.0
duplication,chr22,18704701,18881400,176700,4.102693039635739,0.00015197185093307483
duplication,chr22,19258601,19268300,9700,1.9523681458169895,0.0
duplicatio

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.5571475761015967,0.0
duplication,chr22,2926001,2963000,37000,1.841342671715205,0.0
duplication,chr22,8039001,8048000,9000,2.3628958752481997,0.0
duplication,chr22,13481001,13513000,32000,6.149422412326718,0.0
duplication,chr22,17447001,17811000,364000,1.2742620003723208,8.262306049922465e-05
duplication,chr22,18656001,18683000,27000,1.4011574169192202,0.0003948277563912743
duplication,chr22,18705001,18883000,178000,4.079316656778182,0.0001420755428577333
duplication,chr22,19259001,19268000,9000,1.972101335498861,0.0
duplication,chr22,20326001,20368000,42000,1.4680489545612638,0.00037900609441799825
duplication,chr22,20609001,20655000,46000,1.762314009083093,0.00020788580139976439
duplication,chr22,21466001,21505000,39000,1.52849321044746,0.00017260316962184214
duplication,chr22,21532001,21559000,27000,1.5175891307205125,0.0020322418

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,17450001,17810000,360000,1.2682929130973837,8.352142630150827e-05
duplication,chr22,18650001,18880000,230000,3.364163376409942,0.00016852582927257798
duplication,chr22,21460001,21680000,220000,1.545477432233304,0.00042399366471401626
duplication,chr22,24200001,24240000,40000,1.6846694474208406,6.886056424346341e-05
duplication,chr22,25150001,25310000,160000,1.5318124007459195,0.00011134008167455483
duplication,chr22,28460001,28600000,140000,4.418174293349565,4.558879062816035e-05
duplication,chr22,30800001,30980000,180000,4.493235532009744,1.7284491202193977e-05
duplication,chr22,35940001,36200000,260000,3.1434491709674948,3.306033853786663e-05
duplication,chr22,36470001,36570000,100000,1.68002534442757,8.304498269896193e-05
duplication,chr22,44150001,44470000,320000,3.117389627260297,3.088239422779977e-05
duplication,chr22,46380001,46670000,290000,2.441

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_3_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app